### Kiến trúc mô hình GCN

- 6 lớp GCN (nhằm đảm bảo thông tin được lan truyền giữa các node): Có tác dụng cập nhật embedding của course thông qua các thực thể có liên quan.
- 1 lớp Linear cuối cùng: phân loại chất lượng của khóa học dựa vào embedding của course

### Train GCN model

In [ ]:
!python3 train.py \
    --epochs=5 \
    -lr=1e-3 \
    --grad-clip=1 \
    --val-percent=0.1 \
    --weight-decay=0 \
    --grad-accum=1

python3: can't open file '/home/ptdat/Desktop/graph/src/graph-modeling/graph-modeling/train.py': [Errno 2] No such file or directory


### Demo mô hình GCN

In [2]:
# Load model
import torch
from modeling.modules import HeteroGNN

print("Load lại model đã được huấn luyện")
model = HeteroGNN(
    hidden_channels=128, 
    out_channels=3, 
    num_layers=6
)
model.load_state_dict(torch.load("/home/ptdat/Desktop/graph/checkpoints/epoch-2.pth", weights_only=False))

Load lại model đã được huấn luyện


<All keys matched successfully>

In [3]:
# Ví dụ 1 sample
from modeling.dataset import GraphDataset

dataset = GraphDataset("/home/ptdat/Desktop/graph/samples")
sample = dataset[0]
print("Lấy mẫu 1 đồ thị thuộc dataset:\n")
print(sample)

Lấy mẫu 1 đồ thị thuộc dataset:

{'_id': 'C_677009', 'data': HeteroData(
  course={ x=[1, 2049] },
  field={ x=[0, 1024] },
  resource={ x=[47, 1024] },
  school={ x=[429, 2048] },
  teacher={ x=[4, 2048] },
  user={ x=[3650, 3] },
  comment={ x=[140, 1024] },
  reply={ x=[9, 1024] },
  exercise={ x=[69, 3072] },
  video={ x=[0, 2048] },
  (course, edge, field)={ edge_index=[2, 0] },
  (course, edge, resource)={ edge_index=[2, 47] },
  (course, edge, teacher)={ edge_index=[2, 4] },
  (course, edge, school)={ edge_index=[2, 1] },
  (course, edge, user)={ edge_index=[2, 3650] },
  (course, edge, comment)={ edge_index=[2, 140] },
  (comment, edge, reply)={ edge_index=[2, 9] },
  (user, edge, comment)={ edge_index=[2, 140] },
  (user, edge, reply)={ edge_index=[2, 9] },
  (school, edge, user)={ edge_index=[2, 151] },
  (school, edge, teacher)={ edge_index=[2, 4] },
  (resource, edge, exercise)={ edge_index=[2, 69] },
  (field, rev_edge, course)={ edge_index=[2, 0] },
  (resource, rev_edge,

In [4]:
# Dự đoán 1 course
with torch.inference_mode():
    logits = model(sample["data"].x_dict, sample["data"].edge_index_dict)

idx2label = {
    0: "bad",
    1: "normal",
    2: "good"
}
pred = idx2label[logits[0].argmax().item()]
truth = idx2label[sample["label"].item()]

print(f"Dự đoán: {pred}\nNhãn: {truth}")

Dự đoán: normal
Nhãn: good
